In [3]:
from experiments.pre_train_machine_policy import run_pre_train_machine
from experiments.utils import *
from agent.agents import *
from agent.switching_agents import *
from environments.env import * 
from definitions import ROOT_DIR

from torch.optim import RMSprop
import numpy as np
import random 
import torch
import pickle

np.random.seed(12345678)
random.seed(12345678)
torch.manual_seed(12345678)


## Train baselines and 2-stage algorithm

In [2]:
env_genrator = Environment()
width = 3
height = 20
depth = height//3
init_traffic_level = 'light'
env_params = {'width' : width, 'height':height, 'init_traffic_level': init_traffic_level, 'depth': depth}
env_generator_fn = lambda:env_genrator.generate_grid_world(**env_params)

n_traj = 10000
n_episodes = 5000

human  = NoisyDriverAgent(env_genrator, noise_sd=2.0, noise_sw=2.0, c_H=0.0)
# gather human trajectories
trajectories = gather_human_trajectories(human, env_genrator, n_traj,**env_params)


# define state size 
n_state_features_strings = 1 + depth*(width + 1)

# define state size in 1-hot encoding
n_cell_types = len(env_genrator.cell_types)
n_traffic_levels = len(env_genrator.traffic_levels)
n_state_features_1hot =  n_cell_types + depth*( n_traffic_levels + 1 + width*(n_cell_types + 1)) 

n_state_features = (n_state_features_strings, n_state_features_1hot)

n_actions = 3

lr = 1e-4
optimizer_fn = lambda params: RMSprop(params, lr)

machine = MachineDriverAgent(n_state_features, n_actions, optimizer_fn, c_M=0.2)
fully_automated = FixedSwitchingMachine(n_state_features, optimizer_fn)



with open(f'{ROOT_DIR}/outputs/human_trajectories_{n_traj}', 'wb') as file:
    pickle.dump(trajectories, file, pickle.HIGHEST_PROTOCOL)

1000


NameError: name 'pickle' is not defined

In [5]:
fully_automated, [human, machine] , fully_automated_costs = run_pre_train_machine(fully_automated, [human, machine], trajectories, env_generator_fn, n_episodes)

2021-05-17 19:16:47.324666, Episode 1000, Fully automated on-policy algorithm cumulative cost: 36833.8


In [20]:
from plot.plot_path import PlotPath
from experiments.utils import *
gird_world = env_generator_fn()
plt_path = PlotPath(gird_world, n_try=1)


cost = learn_evaluate(fully_automated, [human, machine] ,gird_world , 
is_learn=False, plt_path = plt_path)
plt_path.plot('../machine_only_light.png')
print(cost)

23.799999999999994
